<a href="https://colab.research.google.com/github/shumshersubashgautam/Pyspark-NLP/blob/main/NGRAM_generator_Wordsegmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Word Segmenter-CHINESE**

WordSegmenterModel-WSM can tokenize non-english texts. Many languages are not whitespace seperated and their sentences are a concationation of many symbols, like Korean, Japanese or Chinese. Withouth understanding the language splitting the Words into their corrosponding tokens is impossible. 
The WordSegmenterModel is trained to understand these languages and split then 
semantically correct.



In [20]:
import os

# Install java
import sparknlp
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start()

In [21]:
import pandas as pd
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

word_segmenter = WordSegmenterModel.pretrained("wordseg_gsd_ud_trad", "zh")\
        .setInputCols(["document"])\
        .setOutputCol("words_segmented")


pipeline = Pipeline(stages=[document_assembler, word_segmenter])
example = spark.createDataFrame(pd.DataFrame({'text': ["""然而，這樣的處理也衍生了一些問題。"""]}))


result = pipeline.fit(example).transform(example)
result.select('words_segmented.result').show()

wordseg_gsd_ud_trad download started this may take some time.
Approximate size to download 1.2 MB
[OK!]


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+----------------------------+
|                      result|
+----------------------------+
|[然而, ，, 這樣, 的, 處理...|
+----------------------------+



# **ENGLISH Chunk Extraction with Chunker **

In [22]:
!pip install -q pyspark==3.3.0  spark-nlp==4.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.7/471.7 kB 37.1 MB/s eta 0:00:00


In [23]:
from pyspark.ml.pipeline import Pipeline

import sparknlp
from sparknlp.annotator import *

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  4.4.1
Apache Spark version:  3.2.3


**Create Spark Dataframe**

In [24]:
!wget -q -O news_category_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/news_Category/news_category_test

In [27]:
!pyspark --packages io.delta:delta-core_2.12:1.1.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"

Python 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b8005120-bcd3-4cbf-aad6-3ccee115dd67;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/1.1.0/delta-core_2.12-1.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;1.1.0!delta-core_2.12.jar (209ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.8/antlr4-runtime-4.8.jar ...
	[SUCC

In [29]:
# import pyspark.sql.functions as F

# news_df = spark.read\ .option("header", "true")\ .csv("/content/news_category_test.csv")\ .withColumnRenamed("description", "text")

# news_df.show(truncate=50)

SyntaxError: ignored

In [ ]:
# # applying POS chunker to find a custom pattern

# chunker = Chunker() \
#             .setInputCols(["document", "pos"]) \
#             .setOutputCol("chunk") \
#             .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

# # NNP: Proper Noun
# # NN: COmmon Noun
# # DT: Determinator (e.g. the)
# # JJ: Adjective

# chunker.extractParamMap()

In [ ]:
# # applying POS chunker to find a custom pattern

# chunker = Chunker() \
#             .setInputCols(["document", "pos"]) \
#             .setOutputCol("chunk") \
#             .setRegexParsers(["<NNP>+", "<DT>?<JJ>*<NN>"])

# # NNP: Proper Noun
# # NN: COmmon Noun
# # DT: Determinator (e.g. the)
# # JJ: Adjective

# chunker.extractParamMap()

# **NGramGenerator**

In [30]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.sql.types import StringType

In [31]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  4.4.1
Apache Spark version:  3.2.3


In [32]:
dfTest = spark.createDataFrame([
    "Cloud computing is benefiting major manufacturing companies",
    "Big data cloud computing cyber security machine learning"
], StringType()).toDF("text")

In [33]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bigrams = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("bigrams") \
            .setN(2)

trigrams_cum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("trigrams") \
            .setN(3)

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    bigrams,
    trigrams_cum
])

In [34]:
# Use the Pipeline in Spark (DataFrame)
model = pipeline.fit(dfTest)
prediction = model.transform(dfTest)

In [35]:
prediction.select("bigrams.result").show(2, truncate=60)

+------------------------------------------------------------+
|                                                      result|
+------------------------------------------------------------+
|[Cloud computing, computing is, is benefiting, benefiting...|
|[Big data, data cloud, cloud computing, computing cyber, ...|
+------------------------------------------------------------+



In [36]:
prediction.select("trigrams.result").show(2, truncate=60)

+------------------------------------------------------------+
|                                                      result|
+------------------------------------------------------------+
|[Cloud computing is, computing is benefiting, is benefiti...|
|[Big data cloud, data cloud computing, cloud computing cy...|
+------------------------------------------------------------+



In [37]:
#Use the Pipeline in Python (string)
from sparknlp.base import LightPipeline

text = 'Cloud computing is benefiting major manufacturing companies'

In [38]:
result = LightPipeline(model).annotate(text)

In [39]:
list(result.keys())

['document', 'token', 'bigrams', 'trigrams']

In [40]:
result['bigrams']

['Cloud computing',
 'computing is',
 'is benefiting',
 'benefiting major',
 'major manufacturing',
 'manufacturing companies']

In [41]:
result['trigrams']

['Cloud computing is',
 'computing is benefiting',
 'is benefiting major',
 'benefiting major manufacturing',
 'major manufacturing companies']